# Recreating "Playing Atari with Deep Reinforcement Learning" paper by DeepMind Technologies

In short this notebook attempts to recreate the Deep Q Network paper by DeepMind. Because DeepMind has computational capabilities that surpass anything a regular individual might have access to some changes are made. The main change to is with an attempt to penalize dying in the hopes of the agent learning to avoid enemy fire. This does not turn out to be beneficial with the amount of testing that was done with the computational power available to me. I ran this notebook iterating over 1000 games while the original paper ran the algorithm for 10 million games. Perhaps with numbers nearing the latter the dying penalty might have had an impact on the behavior.

## Libraries you need

#### OpenAI Gym
You must have both OpenAI Gym and Gym[atari] installed. 

To install: "pip install gym" and "pip install gym[atari]"

### Keras, Tensorflow
Running this is very slow so GPU versions are recommended. However getting the GPU accelerated packages to work is definitely not a trivial thing so feel free to use your CPU! The notebook lets you know if you are running it in GPU mode so it's also a check on if your installation has been done correctly. There is a FORCE_CPU parameter that makes tensorflow use the CPU.

To install: "pip install keras" and "pip install tensorflow"

### Matplotlib
Plotting how the agent learns is fun! 

To install: "pip install matplotlib"

### numba
Use numba to reset the GPU, this can be skipped if not ran on the GPU

To install: "pip install numba"

In [ ]:
import gym
from gym import envs
import numpy as np
import collections
import random
import sys
import matplotlib.pyplot as plt
from numba import cuda

import tensorflow.keras as keras

from tensorflow.python.client import device_lib
import tensorflow as tf

Check to see if tensorflow is indeed running on the GPU and how many GPUs we have available:

In [ ]:
my_devices = device_lib.list_local_devices()
print(my_devices)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

### GPU issues

Sometimes running things on the GPU is a pain and causes issues such as running out of memory, you can just force the notebook to use CPU if you want

In [ ]:
FORCE_CPU = True

As the paper suggests this model can be tested with a bunch of games in the Arcade Learning Environment (ale). Choose here which one to run. More games can easily be added to the list.

In [ ]:
ALE_GAMES = ["SpaceInvaders-v0", "Phoenix-v0","Venture-v0"]
GAME_ID = 0

In [ ]:
# A helper function to reset the GPU as it does have a tendency to get locked in Jupyter
def gpu_reset():
    cuda.select_device(0)
    cuda.close()
# If FORCE_GPU is set on then remove GPUs from devices tensorflow can see
if(FORCE_CPU):
    tf.config.experimental.set_visible_devices(devices= [], device_type='GPU')
else:
    gpu_reset()

Create the environment and record the observation space sizes and the number of possible actions

In [ ]:
GAME_NAME = ALE_GAMES[GAME_ID]
MODEL_NAME = GAME_NAME + "_model"
CONST_LIVES_KEY = 'ale.lives'
env = gym.make(GAME_NAME)
state = env.reset()
obspace = env.observation_space
HEIGHT = obspace.shape[0]
WIDTH = obspace.shape[1]
ACTION_COUNT = env.action_space.n

Helper function - Take an RGB frame and make a BW representation of it. The frame has its data in columns which is then split into pixels each consisting of R,G and B values. The frame, initially HEIGHT x WIDTH, can be cropped to a more suitable size. The original paper first downsamples the frame to 110 x 84 size and then crops it to 84 x 84.

Here we will skip the downsampling and instead crop the image from the center to a rectangle, a square of WIDTH x WIDTH size is a reasonable choice.

In [ ]:
TARGET_HEIGHT = WIDTH
TARGET_WIDTH = WIDTH


TOPCROP = int((HEIGHT - TARGET_HEIGHT)/2)
LEFTCROP = int((WIDTH - TARGET_WIDTH)/2)

def bw_normalize(frame):
    frame = frame/255
    frame = np.mean(frame, 2, keepdims=True)
    return frame[TOPCROP:TOPCROP+TARGET_HEIGHT, LEFTCROP:LEFTCROP+TARGET_WIDTH]

Set the desired behavior for saving and loading models here.

In [ ]:
LOAD = False
SAVE = True

### Learning related hyperparameters. 
Learning happens with the following logic:
A replay buffer is created, containing data for the current frame, action taken, reward received, whether or not the frame was terminal and the next frame. 

After each step the data is recorded and added to the replay buffer with a size of MEMORY_SIZE. The memory buffer will keep its size below MEMORY_SIZE by throwing out the oldest data.

Next a learning step is done. A random sample of SAMPLES is sampled from the replay buffer. The Q network is fit to predict Q(s,a) + r + b max Q'(s', . ), s is the state, a the taken action, b the discount factor, s' the next state, and Q' a network that is periodically updated to match Q. 

Exploration is done with epsilon-greedy strategy with linear epsilon-decay. The initial value of epsilon, rate at which random actions are taken, is initially set to EPSILON_INIT and it then linearly decays with each game towards EPSILON_END reaching it at the final iteration. In the original paper the approach was to train the model with linear epsilon decay for one million games and then for an additional 9 million games with epsilon = 0.1. This is beyond the computational capabilities of normal people and as such we will perform only the epsilon decay portion of the training and with a much smaller number of games.

It is possible to set a penalty for dying, with the hope that this teaches the agent to avoid enemy shots. This requires the last activation layer to be linear as an all-relu network would fail to predict negative values.

Every PERFORMANCE_RECORD_RATE iterations the performance of the model is measured by having it play the game once. As displaiyng the playing slows the performanc down a little, the DRAW_RATE parameter controls how often the performance measuring games are drawn with only one game in DRAW_RATE is drawn.

In [ ]:
WEIGHT_COPY_RATE = 4                                # How often to copy weights from Q' to Q
SAMPLES = 256                                       # How many samples to take from the memory
BATCH_SIZE = 32                                     # Batch size for model.train()
ATTEMPTS = 1000                                     # Number of times to play the game
MEMORY_SIZE = 20000                                 # Size of the memory buffer
SKIPFRAMES = 2                                      # How many frames are skipped between actions / snapshots
GAMMA = 0.99                                        # Learning reward decay
EPSILON_INIT = 1                                    # Initial exploration rate
EPSILON_END = 0.1                                   # Final exploration rate
DIE_PENALTY = 100                                   # Penalize dying. Requires changing the network as ReLUs can't predict negative values!
PERFORMANCE_RECORD_RATE = 5                         # How often to record performance
PERFORMANCE_RECORD_SAMPLES = 5                      # How many samples to take when performance is recorded
DRAW_RATE = 10000                                   # How often do draw performance recording, must be a multiple of PERFORMANCE_RECORD_RATE

### The model

The model is a convolutional neural network with 2convolutional layers followed by a dense layer. The output has ACTION_COUNT neurons and the predicted value is the Q-value associated with each action.

The original paper used 16 8x8 filters with stride 4 followed by a layer of 32 4x4 filters with stride 2.

The input consists of FRAME_COUNT last frames, in the paper this was 4.

In [ ]:
FRAME_COUNT = 4                                      # Number of frames the model will consider, make > 1 to consider past screens too
BORDER_MODE = 'valid'                                # Border mode for convolutions
CONVSIZE_1 = 8                                       # Convolution 1 size
CONVSIZE_2 = 4                                       # Convolution 2 size
STRIDE_1 = 4                                         # Convolution 1 stride
STRIDE_2 = 2                                         # Convolution 2 stride
CONV_FILTERS_1 = 16                                  # Convolution 1 filter count
CONV_FILTERS_2 = 32                                  # Convolution 2 filter count
FINAL_NEURONS = 256                                  # Dense layer neuron count
LEARNING_RATE = 0.001                                # Learning rate, might need hand-tuning

In [ ]:
def construct_network():
    model = keras.models.Sequential()
    model.add(keras.layers.Conv2D(CONV_FILTERS_1, CONVSIZE_1, STRIDE_1, padding = BORDER_MODE, input_shape=(TARGET_HEIGHT, TARGET_WIDTH, FRAME_COUNT)))
    model.add(keras.layers.Activation('relu'))
    model.add(keras.layers.Conv2D(CONV_FILTERS_2, CONVSIZE_2, STRIDE_2, padding = BORDER_MODE))
    model.add(keras.layers.Activation('relu'))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(FINAL_NEURONS))
    # The final activation must be linear if dying penalty is used
    # Otherwise the model breaks as it attempts to predict negative numbers in an all-ReLU network
    # If no dying penalty is used relu is prefered.
    if(DIE_PENALTY>0):
        model.add(keras.layers.Activation('linear'))
    else:
        model.add(keras.layers.Activation('relu'))
    model.add(keras.layers.Dense(ACTION_COUNT))
    model.compile(loss='mse',optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE))
    return model

Depending on the behavior chosen above the model is either loaded or a new one is created

In [ ]:
if(LOAD):
    model = load_model(MODEL_NAME)
else:
    model = construct_network()
model_target = construct_network()

Buffer containing MEMORY_SIZE last frames, actions and rewards represented as its own class

In [ ]:
class ReplayMemory:
    def __init__(self, buffersize):
        self.buffersize = buffersize
        self.count = 0
        self.buffer = collections.deque()

    def add(self, state, action, reward, terminal, nextstate):
        exp = (state, action, reward,terminal, nextstate)
        if(self.count < self.buffersize):
            self.buffer.append(exp)
            self.count += 1
        else:
            self.buffer.popleft()
            self.buffer.append(exp)
            
    def sample(self, size):
        res = []
        if(self.count < size):
            batch = random.sample(self.buffer, self.count)
        else:
            batch = random.sample(self.buffer, size)
        state_batch, action_batch, reward_batch, terminal_batch, nextstate_batch = map(np.array, zip(*batch))
        return state_batch, action_batch, reward_batch, terminal_batch, nextstate_batch

#### Visualizing the play and recording performance

Define functions for running the game with a model or with purely random actions to visualize what the agent has learned so far. The render parameter controls whether or not the game is rendered to a window of if it's run as a background process. 

In [ ]:
def run_game(model, render):
    sys.stdout.write('\r ---------------------------------------------')
    states = collections.deque()
    state = bw_normalize(env.reset())
    for k in range(FRAME_COUNT):
        states.append(state)
    action = 0
    score=0
    newstate = env.step(action)
    skipframe = 0
    while(newstate[2]==False):
        skipframe +=1
        if(skipframe < SKIPFRAMES):
            newstate = env.step(action)
            score += newstate[1]
            continue
        data = np.array(states).reshape(-1,TARGET_HEIGHT,TARGET_WIDTH,FRAME_COUNT)
        actions = model.predict(data)
        action = np.argmax(actions)
        sys.stdout.write('\r'+"Game action: {0}".format(action))
        newstate = env.step(action)
        nextstate = bw_normalize(newstate[0])
        score += newstate[1]
        states.popleft()
        states.append(nextstate)

        if(render):
            env.render()
        skipframe = 0
    sys.stdout.write('\r ---------------------------------------------')
    return score

def run_game_random(render):
    action = 0
    score=0
    newstate = env.step(action)
    skipframe = 0
    while(newstate[2]==False):
        skipframe +=1
        if(skipframe < SKIPFRAMES):
            newstate = env.step(action)
            score += newstate[1]
        action = random.randint(0,ACTION_COUNT-1)
        sys.stdout.write('\r'+"Game action: {0}".format(action))
        newstate = env.step(action)
        nextstate = bw_normalize(newstate[0])
        score += newstate[1]
        states.popleft()
        states.append(nextstate)

        if(render):
            env.render()
        skipframe = 0
    sys.stdout.write('\r ---------------------------------------------')
    return score

#### Learning

Now it's time for the actual learning algorithm. Initialize an array for history so we can record how our agent learned and initialze the ReplayMemory.

In [ ]:
history = []
replaybuffer = ReplayMemory(MEMORY_SIZE)

In [ ]:
for iteration in range(1,ATTEMPTS):
    # Initialize new game, update epsilon and set up buffers
    epsilon =(EPSILON_INIT-EPSILON_END) * (ATTEMPTS-iteration)/ATTEMPTS + EPSILON_END
    sys.stdout.write('\r ---------------------------------------------')
    states = collections.deque()
    state = bw_normalize(env.reset())
    for k in range(FRAME_COUNT):
        states.append(state)
    action = 0
    newstate = env.step(action)
    states.popleft()
    states.append(bw_normalize(newstate[0]))
    frame_number = 1
    skipframe = 0
    reward = 0
    game_ended = False
    lifecounter = newstate[3][CONST_LIVES_KEY]
    old_data = np.array(states).reshape(-1,TARGET_HEIGHT,TARGET_WIDTH,FRAME_COUNT)
    new_data = np.array(states).reshape(-1,TARGET_HEIGHT,TARGET_WIDTH,FRAME_COUNT)
    # Everything initialized, run game loop
    while(game_ended == False):
        # Choose and perform the next action
        frame_number += 1
        skipframe +=1
        # Make decisions only every SKIPFRAMES frame
        if(skipframe > SKIPFRAMES):
            actions = model.predict(new_data)
            action = np.argmax(actions)
            if(random.uniform(0,1)<epsilon):
                action = random.randint(0,ACTION_COUNT-1)
            skipframe = 0
        sys.stdout.write('\r'+"Game iteration: {0}, frame {1}, action {2}".format(iteration,frame_number, action))
        newstate = env.step(action)
        reward += newstate[1]
        if(lifecounter > newstate[3][CONST_LIVES_KEY]):
            reward -= DIE_PENALTY
        lifecounter = newstate[3][CONST_LIVES_KEY]
        
        # Update the states      
        nextscreen = bw_normalize(newstate[0])
        states.popleft()
        states.append(nextscreen)
                
        # Store the transition
        old_data = new_data
        new_data = np.array(states).reshape(-1,TARGET_HEIGHT,TARGET_WIDTH,FRAME_COUNT)
        game_ended = newstate[2]
        replaybuffer.add(old_data, action, reward, not game_ended, new_data)
        
    sys.stdout.write('\r ---------------------------------------------')
    sys.stdout.write('\r Training')
    # Random mini batch from replaybuffer and fit
    state_batch, action_batch, reward_batch, terminal_batch, newdata_batch = replaybuffer.sample(SAMPLES)
    batchsize_actual = len(newdata_batch)
    state_batch = state_batch.reshape(batchsize_actual, TARGET_HEIGHT, TARGET_WIDTH, FRAME_COUNT)
    newdata_batch = newdata_batch.reshape(batchsize_actual, TARGET_HEIGHT, TARGET_WIDTH, FRAME_COUNT)
    # Update only new information
    targets = model_target.predict(state_batch)
    # Use old estimation for future value
    predictions = model.predict(newdata_batch)
    # Make sure the model is not broken and predicting NaN values
    a = np.sum(targets.reshape(-1))
    b = np.sum(predictions.reshape(-1))
    if(np.isnan(a) or np.isnan(b)):
        sys.stdout.write('\r ERROR: Broken model - too high learning rate?')
        break
    for i in range(len(action_batch)):
        targets[i,action_batch] = reward_batch[i] + int(terminal_batch[i]) * GAMMA * np.max(predictions[i])
    model_target.fit(state_batch.reshape(batchsize_actual, TARGET_HEIGHT, TARGET_WIDTH, FRAME_COUNT), targets, batch_size=BATCH_SIZE, verbose=0)
    if(iteration % WEIGHT_COPY_RATE == 0):
        model.set_weights(model_target.get_weights())
    sys.stdout.write('\r Measuring performance')
    if(iteration % PERFORMANCE_RECORD_RATE==0):
        render = False
        if(iteration % DRAW_RATE == 0):
            render = True
        evaluated = []
        for measurement in range(PERFORMANCE_RECORD_SAMPLES):
            evaluated += [run_game(model, render)]
        history += [[iteration, np.mean(evaluated)]]

In [ ]:
if(SAVE):
    model.save(MODEL_NAME)

In [ ]:
x = []
y = []
for h in history:
    x += h[0]
    y += h[1]
plt.plot(x,y)

### Visualizing the resulting model

We can now compare the trained model to a random model

In [ ]:
env.reset()
run_game(model, True)

In [ ]:
env.reset()
run_game_random(True)

### What's next?

This notebook added the concept penalizing dying in the attempt to have the agent learn to dodge. However this did not quite work as expected and could certainly be improved upon. More iterations would most likely help, but perhaps there is a better strategy.